This script will take a model I saved and upload it to AI platform for earth engine.  Note the model must be sent to a google drive location manually first.  I actually do this with a colab script called eeify_trained_good.ipynb


In [ ]:
from google.colab import auth
auth.authenticate_user()
import ee
ee.Authenticate()
ee.Initialize(project = 'gee-serdp-upload')

from google.colab import drive
drive.mount('/content/drive')

Location of model in google drive

In [ ]:
# model_path_tf = '/content/drive/MyDrive/l8_sent_collection2_079_128.tf'
# model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_0_128.tf"
# model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_0_global_norm.tf"
# model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_0_3_norm_cutoff.tf"
# model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_0_3_direction.tf"
# model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_0_3_norm_cutoff_512.tf"
# model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_crop_dnbr_0_two_long_128_crop_mtn.tf"
# model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_crop_VI_0_two_128.tf"
# model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_dnbr_full_norm.tf"
# model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_crop_dnbr_0_two_long_128_crop_mtn.tf"
# model_path_tf = "/content/drive/MyDrive/l8_collection2_dnbr_five_128_2d_ds.tf"
# model_path_tf = "/content/drive/MyDrive/l8_collection2_dnbr_one_128_2d_ds_proj_mtbs.tf"
# model_path_tf = "/content/drive/MyDrive/modis_dnbr_0_one_plus_128_2d_ds.tf"
# model_path_tf = '/content/drive/MyDrive/l8_collection2_VI_one_128_2d_ds_proj_final_mtbs_gids.tf'
# model_path_tf = '/content/drive/MyDrive/nbac_mtbs_2015_2019.tf'
# model_path_tf = '/content/drive/MyDrive/russia_good_2015_2019.tf'
model_path_tf = '/content/drive/MyDrive/combined_good_2015_2019.tf'



# l8_collection2_VI_one_128_2d_ds_proj_final_mtbs.tf

# model_path_tf = "/content/drive/MyDrive/modis_dnbr_0_five_plus_128_3d_crop_no_ds.tf"








# model.save(model_path_tf, save_format='tf')

Steps to upload to AI platform

In [ ]:
from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(model_path_tf, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: "output"}) + "'"
print(input_dict)
print(output_dict)

# Put the EEified model next to the trained model directory.
# EEIFIED_DIR = '/content/drive/MyDrive/EmptyUnet2_test_eeify.tf'
# EEIFIED_DIR = '/content/drive/MyDrive/l8_sent_collection2_079_128_eeify.tf'
# EEIFIED_DIR = '/content/drive/MyDrive/l8_sent_collection2_0_128_eeify.tf'
# EEIFIED_DIR = '/content/drive/MyDrive/l8_sent_collection2_0_global_norm_eeify.tf'
# EEIFIED_DIR = '/content/drive/MyDrive/l8_sent_collection2_0_3_direction_eeify.tf'
# EEIFIED_DIR = 'content/drive/MyDrive/l8_sent_collection2_0_3_norm_cutoff_512_eeify.tf'
# EEIFIED_DIR = 'content/drive/MyDrive/l8_collection2_VI_one_128_2d_ds_proj_final_mtbs_gids_eeify'
# EEIFIED_DIR = 'content/drive/MyDrive/nbac_mtbs_2015_2019_eeify'
# EEIFIED_DIR = 'content/drive/MyDrive/russia_good_2015_2019_eeify'
EEIFIED_DIR = 'content/drive/MyDrive/combined_good_2015_2019_eeify'


# EEIFIED_DIR = 'content/drive/MyDrive/russia_eeify'

# shapefile.transform(targetProjection);
PROJECT = 'gee-serdp-upload'

!gcloud config set project {PROJECT}
# You need to set the project before using the model prepare command.
!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {model_path_tf} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

%%writefile config.yaml
autoScaling:
  minNodes: 10

REGION = 'us-east4'
# MODEL_NAME = 'Empty_Unet2_test'
# MODEL_NAME = 'l8_sent_collection2_079_128_eeify_test'
# MODEL_NAME = 'l8_sent_collection2_0_128_eeify_test2'
# MODEL_NAME = 'l8_sent_collection2_0_global_norm_eeify'
# MODEL_NAME = 'l8_sent_collection2_0_3_direction_eeify'
# MODEL_NAME = 'l8_sent_collection2_0_3_norm_cutoff_512_eeify'
# MODEL_NAME = 'l8_collection2_dnbr_one_128_2d_ds_proj_mtbs_eeify'
# MODEL_NAME = 'nbac_mtbs_2015_2019_eeify'

# MODEL_NAME = 'russia_good_2015_2019_eeify'
MODEL_NAME = 'combined_good_2015_2019_eeify'

VERSION_NAME = 'v0'
STAGING_BUCKET = 'gs://smp-scratch'

!gcloud ai-platform models create {MODEL_NAME} \
  --project {PROJECT} \
  --region {REGION}

!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --region {REGION} \
  --model {MODEL_NAME} \
  --origin {EEIFIED_DIR} \
  --staging-bucket {STAGING_BUCKET} \
  --framework "TENSORFLOW" \
  --runtime-version=2.5 \
  --python-version=3.7
